# NEURO-105: Statistics and Probability using Python
## Lesson 3 - Monday 26/1/26

**Instructor:** Alexandros Pittis  
**Course:** MSc in Neurosciences, University of Crete

---

### Objectives
1. Loading real data from CSV files
2. Understanding p-values
3. Comparing two groups: t-test
4. Non-parametric alternative: Mann-Whitney U test

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

print("Libraries loaded!")

---
## Part 1: Loading Real Data

Today we'll work with **gene expression data** from a Parkinson's disease study.

The data contains:
- 20 brain tissue samples (globus pallidus)
- 10 healthy controls, 10 Parkinson's patients
- Expression levels of 15 genes related to neurodegeneration

**Data source:** Based on GEO dataset GSE20146

### Loading a CSV file

CSV = Comma Separated Values - a simple text format for tabular data.

In [ ]:
# Load data from a URL (hosted on GitHub)
url = "https://raw.githubusercontent.com/cgenomicslab/Courses/refs/heads/main/MScNeuro/2026/data/parkinsons_expression.csv"

data = pd.read_csv(url)

data

### Alternative: Load from a local file

If you have the file downloaded:
```python
data = pd.read_csv("parkinsons_expression.csv")
```

Or with full path:
```python
data = pd.read_csv("/path/to/your/file.csv")
```

### Exploring the data

In [ ]:
# Shape: (rows, columns)
data.shape

In [ ]:
# Column names
data.columns

In [ ]:
# Data types
data.dtypes

In [ ]:
# How many samples per condition?
data['condition'].value_counts()

In [ ]:
# Summary statistics
data.describe()

### Genes' info

| Gene | Function |
|------|----------|
| SNCA | Alpha-synuclein - aggregates in Parkinson's |
| PARK2 | Parkin - mutations cause PD |
| PINK1 | Sensor for mitochondrial damage |
| TH | Tyrosine hydroxylase - dopamine biosynthesis enzyme |
| SLC6A3 | Dopamine transporter |
| BDNF | Brain-derived neurotrophic factor |

---
## Part 2: What is a P-value?

Before we do any tests, let's see what we're calculating.

### The Question

We have two groups (Control vs PD). They have different mean expression levels.

**Is this real difference, or just random chance?**

In [ ]:
# Look at SNCA expression in both groups
control = data[data['condition'] == 'Control']['SNCA']
pd_group = data[data['condition'] == 'PD']['SNCA']

print("Control mean:", round(control.mean(), 2))
print("PD mean:", round(pd_group.mean(), 2))
print("Difference:", round(pd_group.mean() - control.mean(), 2))

In [ ]:
# Visualize the difference
sns.boxplot(data=data, x='condition', y='SNCA')
plt.title('SNCA Expression: Control vs Parkinson\'s')

### The P-value Concept

**P-value** = The probability of seeing a difference this large (or larger) **if there was in fact no real difference** between the groups.

- Assuming the "null hypothesis": there is NO difference between groups
- P-value tells us: how likely would we observe our data under this assumption?

**Interpretation:**
- Small p-value (< 0.05) → "The difference is unlikely to be just chance" → **Statistically significant**
- Large p-value (> 0.05) → "The difference could be explained by random chance" → **Not significant**

### Visually

Imagine two overlapping distributions. Even if they come from the **same** population, random sampling will give slightly different means.

In [ ]:
# SNCA expression - separated distributions
sns.kdeplot(data=data, x='SNCA', hue='condition', fill=True, alpha=0.5)
plt.title('SNCA: Clear separation → likely significant')

In [ ]:
# COMT expression - overlapping distributions  
sns.kdeplot(data=data, x='COMT', hue='condition', fill=True, alpha=0.5)
plt.title('COMT: Large overlap → probably not significant')

---
## Part 3: The t-test

The **t-test** compares the means of two groups and gives us a p-value.

**Assumptions:**
- Data is approximately normally distributed
- Groups have similar variance
- Observations are independent

### Independent samples t-test

Use when comparing two **independent** groups (like Control vs PD).

In [ ]:
# t-test for SNCA
control_snca = data[data['condition'] == 'Control']['SNCA']
pd_snca = data[data['condition'] == 'PD']['SNCA']

t_stat, p_value = stats.ttest_ind(control_snca, pd_snca)

print("t-statistic:", round(t_stat, 3))
print("p-value:", p_value)

**Interpretation:**
- p-value is small (< 0.05)
- The difference in SNCA expression between Control and PD is **statistically significant**
- SNCA is upregulated in Parkinson's disease

In [ ]:
# t-test for COMT (less difference expected)
control_comt = data[data['condition'] == 'Control']['COMT']
pd_comt = data[data['condition'] == 'PD']['COMT']

t_stat, p_value = stats.ttest_ind(control_comt, pd_comt)

print("t-statistic:", round(t_stat, 3))
print("p-value:", round(p_value, 4))

**Interpretation:**
- p-value is larger (closer to or above 0.05)
- The difference in COMT expression is **marginally statistically significant**

### Testing multiple genes

In [ ]:
# Get list of gene columns (exclude sample_id and condition)
genes = data.columns[2:]

print("Genes:", list(genes))

In [ ]:
# Run t-test for each gene
results = []

for gene in genes:
    control = data[data['condition'] == 'Control'][gene]
    pd_group = data[data['condition'] == 'PD'][gene]
    
    t_stat, p_val = stats.ttest_ind(control, pd_group)
    
    results.append({
        'gene': gene,
        'control_mean': round(control.mean(), 2),
        'pd_mean': round(pd_group.mean(), 2),
        'difference': round(pd_group.mean() - control.mean(), 2),
        'p_value': p_val
    })

# Create results table
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('p_value')
results_df

In [ ]:
# Which genes are significantly different? (p < 0.05)
significant = results_df[results_df['p_value'] < 0.05]
print("Significantly different genes:")
significant

---
## Part 4: Mann-Whitney U Test

The **Mann-Whitney U test** (also called Wilcoxon rank-sum test) is a **non-parametric** alternative to the t-test.

**When to use:**
- Data is not normally distributed
- Small sample sizes
- Ordinal data (rankings)
- Outliers present

**How it works:**
- Ranks all values from both groups together
- Compares the sum of ranks between groups
- Does NOT assume normal distribution

In [ ]:
# Mann-Whitney U test for SNCA
control_snca = data[data['condition'] == 'Control']['SNCA']
pd_snca = data[data['condition'] == 'PD']['SNCA']

u_stat, p_value = stats.mannwhitneyu(control_snca, pd_snca)

print("U-statistic:", u_stat)
print("p-value:", p_value)

### Comparing t-test vs Mann-Whitney

In [ ]:
# Compare both tests for all genes
comparison = []

for gene in genes:
    control = data[data['condition'] == 'Control'][gene]
    pd_group = data[data['condition'] == 'PD'][gene]
    
    _, p_ttest = stats.ttest_ind(control, pd_group)
    _, p_mannwhitney = stats.mannwhitneyu(control, pd_group)
    
    comparison.append({
        'gene': gene,
        'p_ttest': round(p_ttest, 4),
        'p_mannwhitney': round(p_mannwhitney, 4)
    })

comparison_df = pd.DataFrame(comparison)
comparison_df

- For most genes, both tests give similar results
- Small differences are normal
- If results differ dramatically, investigate your data distribution

### When to use which test?

| Situation | Use |
|-----------|-----|
| Normal data, large samples (n > 30) | t-test |
| Non-normal data | Mann-Whitney |
| Small samples (n < 30) | Mann-Whitney (safer) |
| Ordinal data (rankings) | Mann-Whitney |
| Unsure about distribution | Mann-Whitney |

---
## Part 5: Visualizing Results

In [ ]:
# Box plots for top significant genes
top_genes = ['SNCA', 'TH', 'PARK2', 'PINK1']

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()

for i, gene in enumerate(top_genes):
    sns.boxplot(data=data, x='condition', y=gene, ax=axes[i])
    
    # Get p-value
    ctrl = data[data['condition'] == 'Control'][gene]
    pd_g = data[data['condition'] == 'PD'][gene]
    _, p = stats.ttest_ind(ctrl, pd_g)
    
    axes[i].set_title(gene + ' (p = ' + str(round(p, 4)) + ')')

plt.tight_layout()
plt.show()

---
## Part 6: Summary

**Loading data:**
- `pd.read_csv(url)` or `pd.read_csv("file.csv")`

**P-value:**
- Probability of seeing the observed difference if there was no real effect
- p < 0.05 → statistically significant
- p > 0.05 → not significant

**t-test:**
- `stats.ttest_ind(group1, group2)` for independent samples
- Assumes normal distribution
- Returns (t-statistic, p-value)

**Mann-Whitney U test:**
- `stats.mannwhitneyu(group1, group2)`
- Non-parametric (no normality assumption)
- Safer for small samples or non-normal data

---
## Exercises

### Exercise 1
Perform a t-test on the BDNF gene. Is the difference significant?

In [ ]:
# YOUR CODE HERE


### Exercise 2
Create a box plot for the TH gene (Tyrosine hydroxylase). Add the p-value to the title.

In [ ]:
# YOUR CODE HERE


### Exercise 3
Compare the t-test and Mann-Whitney U test results for the LRRK2 gene. Do they agree?

In [ ]:
# YOUR CODE HERE


---
## Resources

- [SciPy t-test documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
- [SciPy Mann-Whitney documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html)
- [Understanding p-values](https://www.nature.com/articles/nmeth.4210)
- [GEO Database](https://www.ncbi.nlm.nih.gov/geo/) - source of original data

---

**Next class (28/1/26):** Hands-on project, case study

---
*NEURO-105 - MSc in Neurosciences, University of Crete*